# DVC Options - Metrics and Plots
## The DVC way

In the following part we will look into metrics and plots from ZnTrack Nodes.
All `dvc run` options listed [here](https://dvc.org/doc/command-reference/run#options) can be used via `dvc.<option>`.
With the exception of params, which is handled automatically.
All these options take either `str` or `pathlib.Path` directed to the file the content should be stored in.
As shown before, `dvc.deps` can also take another `Node` as an argument.

In [1]:
from zntrack import Node, dvc, config
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [2]:
config.nb_name = "04_metrics_and_plots.ipynb"

In [3]:
from zntrack.utils import cwd_temp_dir
temp_dir = cwd_temp_dir()

In [4]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmp_1wtk6lf/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In the following we define a simple Node that produces a metric and a plot output using `json` and `pandas`.
We will queue multiple experiments with different outputs and then compare them afterwards.
With `@Node(silent=True)` we can reduce the amount of logs that will be displayed.

In [5]:
@Node(silent=True)
class MetricAndPlot:
    my_metric: Path = dvc.metrics(Path("my_metric.json"))
    my_plots: Path = dvc.plots("my_plots.csv")
    pre_factor = dvc.params()
    
    def __call__(self, pre_factor=1.0):
        self.pre_factor = pre_factor
    
    def run(self):
        self.my_metric.write_text(
            json.dumps({
                "metric_1": 17 * self.pre_factor,
                "metric_2": 42 * self.pre_factor
            })
        )
        
        x_data = np.linspace(0, 1.0 * self.pre_factor, 1000)
        y_data = np.exp(x_data)
        df = pd.DataFrame({"y": y_data, "x": x_data}).set_index("x")
        
        df.to_csv(self.my_plots)

In [6]:
metric_and_plot = MetricAndPlot()
metric_and_plot()
!dvc repro
!git add .
!git commit -m "First Run"

Running stage 'MetricAndPlot':                                        core>
> python3 -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot(load=True, name='MetricAndPlot').run()" 
Generating lock file 'dvc.lock'                                                 
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.
[master (root-commit) 5b00fe2] First Run
 17 files changed, 1352 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/confusion_normalized.json
 create mode 100644 .dvc/plots/linear.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/simple.json
 create mode 100644 .dvc/plots/smooth.json
 create mode 100644 .dvcignore
 create mode 100644 .gitignore
 create mode 100644 .zntrack.json
 create mode 100644 04_metrics_and_plots.ipynb
 create mode 100644 dv

In [7]:
metric_and_plot(pre_factor=2)
!dvc exp run --queue --name "factor_2"
metric_and_plot(pre_factor=3)
!dvc exp run --queue --name "factor_3"
metric_and_plot(pre_factor=4)
!dvc exp run --queue --name "factor_4"
metric_and_plot(pre_factor=5)
!dvc exp run --queue --name "factor_5"

Queued experiment '2a5ab00' for future execution.                     core>
Queued experiment '63fd395' for future execution.                     core>
Queued experiment 'f56fb6b' for future execution.                     core>
Queued experiment '71793fc' for future execution.                     core>


In [8]:
!dvc exp run --run-all -j 4

  0% Checkout|                                       |0/2 [00:00<?,     ?file/s]
!
  0%|          |.XciYvsyqeocYfnuJHy95Fy.tmp          0/36 [00:00<?,       ?it/s]
  0% Checkout|                                       |0/2 [00:00<?,     ?file/s]
!
!
! Checkout|                                       |0/2 [00:00<?,     ?file/s]

  0%|          |.Pkb4u6io6kkVJMqDaxnRie.tmp          0/36 [00:00<?,       ?it/s]
                                                                                
!
  0%|          |.Pr7BDWvmxzGoRLv7r46yi2.tmp          0/36 [00:00<?,       ?it/s]
                                                                                
!
  0%|          |.dA7MUZY9Mu3CX3qXqsahJ5.tmp    0.00/38.0k [00:00<?,       ?it/s]
                                                                                

!

  0%|          |.VzomkVd22D9Cchg9bXxFVi.tmp    0.00/38.0k [00:00<?,       ?it/s]
                                                                                
                

Now that all experiments are done, we can look at the metrics directly with `dvc exp show` or `dvc metrics show/diff`

In [9]:
!dvc exp show --csv --no-timestamp > exp_show.csv
pd.read_csv("exp_show.csv", index_col=0)

,rev,typ,parent,metric_1,metric_2,MetricAndPlot.pre_factor
Experiment,,,,,,
NaN,workspace,baseline,NaN,17.0,42.0,5.0
master,5b00fe2,baseline,NaN,17.0,42.0,1.0
factor_4,41310f8,branch_commit,NaN,68.0,168.0,4.0
factor_2,a0e84e0,branch_commit,NaN,34.0,84.0,2.0
factor_5,05cf9c0,branch_commit,NaN,85.0,210.0,5.0
factor_3,b2dfa49,branch_base,NaN,51.0,126.0,3.0


We can also use `dvc plots show/diff` to evaluate the plot data that we produced.

In [10]:
!dvc plots diff HEAD factor_2 factor_3 factor_4 factor_5

file:///tmp/tmp_1wtk6lf/dvc_plots/index.html


In [11]:
from IPython.display import IFrame

In [12]:
IFrame(src='../../_static/plots/overview_04_plots.html', width=700, height=600)

## The ZnTrack way

ZnTrack provides and easier way to handle metrics. Similar to `zn.outs()` which does not require defining a path to outs file, one can use `zn.metrics`.

In [13]:
from zntrack import zn

@Node(silent=True)
class ZnTrackMetric:
    my_metric = zn.metrics()
    
    def run(self):
        self.my_metric = {"alpha": 1.0, "beta": 0.00473}
        
ZnTrackMetric()()

In [14]:
!dvc repro

Running stage 'ZnTrackMetric':                                        core>
> python3 -c "from src.ZnTrackMetric import ZnTrackMetric; ZnTrackMetric(load=True, name='ZnTrackMetric').run()" 
Updating lock file 'dvc.lock'                                                   

Stage 'MetricAndPlot' is cached - skipping run, checking out outputs
Updating lock file 'dvc.lock'                                                   

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [15]:
!dvc exp show --csv --no-timestamp > exp_show.csv
pd.read_csv("exp_show.csv", index_col=0)

,rev,typ,parent,metric_1,metric_2,my_metric.gamma,my_metric.beta,MetricAndPlot.pre_factor
Experiment,,,,,,,,
NaN,workspace,baseline,NaN,85.0,210.0,1.0,0.00473,5.0
master,5b00fe2,baseline,NaN,17.0,42.0,1.0,NaN,NaN
factor_4,41310f8,branch_commit,NaN,68.0,168.0,4.0,NaN,NaN
factor_2,a0e84e0,branch_commit,NaN,34.0,84.0,2.0,NaN,NaN
factor_3,b2dfa49,branch_commit,NaN,51.0,126.0,3.0,NaN,NaN
factor_5,05cf9c0,branch_base,NaN,85.0,210.0,5.0,NaN,NaN


In [16]:
temp_dir.cleanup()